In [1]:

import Thymio as th
import time
import videopath1 as vp
import numpy as np
import threading
import vision.detect_aruco as da
import queue
import cv2
def draw_path(img, path):
    for i in range(len(path)-1):
        cv2.line(img, path[i], path[i+1], (0, 255, 0), 2)
    return img
def query_vars(stop:queue.Queue,thymio:th.Thymio):
    while True and stop.empty():
        thymio.wait_for_variables()
    
        
        
        

In [ ]:

channel = queue.Queue(maxsize=2)
stop = queue.Queue(maxsize=1)
aruco_thread = threading.Thread(target=da.main,args=(channel,stop), daemon=True)
aruco_thread.start()
print("Open thymio")
thymio = th.Thymio()
await thymio.initiateLock()
print("Thymio opened")
query = channel.get()



camera_on = query[0]
original_image = query[1]
display_image = query[2]

start = query[3]
print(f"start: {start}")
cv2.imshow("mainframe", display_image)
cv2.waitKey(1)

while not camera_on or start == (0,0,0):
    query = channel.get()
    camera_on = query[0]
    original_image = query[1]
    display_image = query[2]
    start = query[3]
    cv2.imshow("mainframe", display_image)
    cv2.waitKey(1)
    print("waiting for camera", end="\r")

print("first pos queried",start)
path_cm, path, costmap, block_height, block_width, start, goal, display_image, cm_per_pixel =vp.init(original_image,start)
#cv2.imshow("Select Goal", original_image)
#cv2.setMouseCallback("Select Goal", vp.select_points, original_image)
cm_p_pixel = 77/block_width
#path = vp.update(costmap,block_height,block_width,start,vp.points[0],original_image,cm_p_pixel,[])
next_goal = path[0]
current_pos = np.array([start[0],start[1],start[2]])
current_speed = np.array([0,0,0])
camera_position = start
z_predicted = np.concatenate([current_pos,current_speed]) #kalman_state
sigma_predicted = np.diag([0.1,0.1,0.1,0,0,0])

z_list = []

vars_towait = ['prox.horizontal','motor.right.speed','motor.left.speed','motor.right.target','motor.right.target']
#vars_thread = threading.Thread(target=query_vars, args=(stop,thymio), daemon=True)
#vars_thread.start()
time.sleep(1)
path = path[2:]
thetas = (0.0,0.0)
targets = (0.0,0.0)
while True:
    thymio.wait_for_variables(vars_towait)
    query = channel.get()
    camera_on = query[0]
    #original_image = query[1]
    display_image = query[2]
    start = query[3]

    #draw_path(display_image, path_cm)
    cv2.putText(display_image, f"X: {z_predicted[0]:.2f} Y: {z_predicted[1]:.2f} Theta: {z_predicted[2]:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(display_image, f"maxmin theta {int(thetas[0]), int(thetas[1])}, targets:{int(targets[0]), int(targets[1])}", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    #draw the grid on the image
    
    #for i in range(0,display_image.shape[1],int(cm_per_pixel)):
    #    cv2.line(display_image, (i,0), (i,display_image.shape[0]), (255, 0, 0), 1)
    #for i in range(0,display_image.shape[0],int(cm_per_pixel)):
    #    cv2.line(display_image, (0,i), (display_image.shape[1],i), (255, 0, 0), 1)
        

    cv2.imshow("mainframe", display_image)
    cv2.waitKey(1)
    print(path, end="\r")
    next_goal = path[0]
    print(f'Next goal : {next_goal}')
    print(f'Current position : {current_pos}')
    translation_or_rotation, thetas, targets = thymio.navigate(current_pos, next_goal)
    current_pos = z_predicted[:3] # Is used for the controller
    speedright = thymio.getSpeedR()
    speedleft = thymio.getSpeedL()
    
    
    z_filtered, sigma_filtered= thymio.filtering_step(z_k_k_1 = z_predicted, 
                                    sigma_k_k_1 = sigma_predicted, 
                                    V_left_measure =speedleft, 
                                    V_right_measure = speedright, 
                                    camera_working=camera_on, 
                                    x_measured=camera_position[0], 
                                    y_measured=camera_position[1],
                                    theta_measured= camera_position[2],
                                    translation_or_rotation = translation_or_rotation)
    
    z_list.append(z_filtered)
    
    
    z_predicted, sigma_predicted = thymio.prediction_step(z_filtered, sigma_filtered)
    # put the zpredicted on the image
    
    obstacle,detected = thymio.getObstaclePosition()
    if detected:
        print("Obstacle detected")
        #TODO: use the obstacle position to update the costmap
        original_image = query[1]
        path,costmap = vp.update(costmap,block_height,block_width,current_pos,vp.points[0],original_image,cm_p_pixel,obstacle)
        
    if thymio.robot_close_waypoint(current_pos[:2], next_goal[0], next_goal[1]):
        path.pop(0)
        if len(path) == 0:
            print("Goal reached")
            thymio.set_multiple_variables({"motor.left.target":[0],"motor.right.target":[0]})
            #vars_thread.join()
            thymio.stop()
            aruco_thread.join()
            break
        
        

Open thymio
[INFO] detecting 'DICT_ARUCO_ORIGINAL' markers...
Thymio opened
start: (48.14237594604492, 306.5103454589844, 0.1707352114752828)(121, 36) pos2 (1127, 42) pos3 (1193, 570) pos4 (47, 544) pos5 [ 48.142376 306.51035 ], scaled (266.6666666666667, 2.0833333333333335), width 3, height720 
first pos queried (48.14237594604492, 306.5103454589844, 0.1707352114752828)os2 (1128, 41) pos3 (1193, 570) pos4 (47, 544) pos5 [ 48.142376 306.51035 ], scaled (266.6666666666667, 2.0833333333333335), width 3, height720 
The costmap shape is: (8, 15) ]]], pos5 shape = (1, 1, 2) /rpos1 (121, 36) pos2 (1128, 41) pos3 (1193, 570) pos4 (47, 544) pos5 [ 48.142376 306.51035 ], scaled (266.6666666666667, 2.0833333333333335), width 3, height720 
Click to select the goal point.]], pos5 shape = (1, 1, 2) /rpos1 (122, 36) pos2 (1128, 41) pos3 (1193, 570) pos4 (47, 544) pos5 [ 48.142376 306.51035 ], scaled (266.6666666666667, 2.0833333333333335), width 3, height720 
Goal point selected at: (870, 483) pos5 

In [ ]:
import cv2
print(cv2.__version__)

4.6.0
